#### Ejercicio 2

In [1]:
import org.apache.spark.sql.DataFrame;
import org.apache.spark.util.SizeEstimator;
import org.apache.spark.sql.functions.desc;

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.33:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1616621481933)
SparkSession available as 'spark'


import org.apache.spark.sql.DataFrame
import org.apache.spark.util.SizeEstimator
import org.apache.spark.sql.functions.desc


In [4]:
val DataScheme: DataFrame = spark.read.parquet("processed");

DataScheme: org.apache.spark.sql.DataFrame = [campaign_start_date: date, personal_type: string ... 4 more fields]


In [6]:
class EvaluateDataFrame (val dataScheme: DataFrame,
                         val PrmSparkStatsLevel: Int,
                         val sparkPartitionId: String) {
    val dfPartitions = dataScheme.rdd.getNumPartitions
    require(PrmSparkStatsLevel > 0, """Spark level must be
    greater than 0""")
}

defined class EvaluateDataFrame


In [92]:
//Main super trait
trait PrmLevel1 {
    def toMb (num: Float): Float = num / 1024.toFloat / 1024.toFloat
    def dfStats(df: EvaluateDataFrame) = {
        val dfSizeByte: BigInt = df.dataScheme.queryExecution.optimizedPlan.stats.sizeInBytes
        val dfSizeMb: Float = toMb(dfSizeByte.toFloat)
        val dfSizeEstimatorMb: Float = toMb(SizeEstimator.estimate(df.dataScheme))
        println("*** STATS *** Df Stats PlanSizeMB:"+dfSizeMb+",  EstimatorSizeMB:"
                +dfSizeEstimatorMb+", PartitionNum: "+df.dfPartitions+", PartitionSizeMB:"
                +dfSizeMb / df.dfPartitions+", PartitionSizeMB:"+dfSizeEstimatorMb / df.dfPartitions)
    }
}

defined trait PrmLevel1


In [99]:
trait PrmLevel2 extends PrmLevel1 {
    override def dfStats(df: EvaluateDataFrame) = {
        super.dfStats(df);
        println(s"""*** STATS *** Df Partition record 
        count ${df.dataScheme.count()}""");
    }
}

defined trait PrmLevel2


In [15]:
trait PrmLevel3 extends PrmLevel1 {
    override def dfStats(df: EvaluateDataFrame) = {
        super.dfStats(df);
        println(s"*** STATS *** Df Partition Info");
        df.dataScheme
          .groupBy(df.sparkPartitionId)
          .count()
          .orderBy(desc("count"))
          .show(if (df.dfPartitions>0) df.dfPartitions else 200);
    }
}

defined trait PrmLevel3


In [17]:
trait PrmLevel8 extends PrmLevel1 {
    override def dfStats(df: EvaluateDataFrame) = {
        super.dfStats(df);
        println(s"*** STATS *** Show data:")
        df.dataScheme.show() 
    }
}

defined trait PrmLevel8


In [84]:
class PrmSparkStatsLevel1 (val df: EvaluateDataFrame) extends PrmLevel1 {
    super.dfStats(df)
}

defined class PrmSparkStatsLevel1


In [57]:
class PrmSparkStatsLevel2 (val df: EvaluateDataFrame) extends PrmLevel2 {
    super.dfStats(df)
}

defined class PrmSparkStatsLevel2


In [58]:
class PrmSparkStatsLevel3 (val df: EvaluateDataFrame) extends PrmLevel3 {
    super.dfStats(df)
}

defined class PrmSparkStatsLevel3


In [61]:
class PrmSparkStatsLevel4 (val df: EvaluateDataFrame) extends PrmLevel2 with PrmLevel3 {
    super.dfStats(df)
}

defined class PrmSparkStatsLevel4


In [59]:
class PrmSparkStatsLevel8 (val df: EvaluateDataFrame) extends PrmLevel8 {
    super.dfStats(df)
}

defined class PrmSparkStatsLevel8


In [101]:
val df1: EvaluateDataFrame = new EvaluateDataFrame(DataScheme, 1, "cutoff_date")
val df2: EvaluateDataFrame = new EvaluateDataFrame(DataScheme, 2, "cutoff_date")
val df3: EvaluateDataFrame = new EvaluateDataFrame(DataScheme, 3, "cutoff_date")
val df4: EvaluateDataFrame = new EvaluateDataFrame(DataScheme, 4, "cutoff_date")
val df8: EvaluateDataFrame = new EvaluateDataFrame(DataScheme, 8, "cutoff_date")

df1: EvaluateDataFrame = EvaluateDataFrame@46c777eb
df2: EvaluateDataFrame = EvaluateDataFrame@5103f8dd
df3: EvaluateDataFrame = EvaluateDataFrame@1d372d21
df4: EvaluateDataFrame = EvaluateDataFrame@797246d4
df8: EvaluateDataFrame = EvaluateDataFrame@4771e3e


In [103]:
val prm1 = new PrmSparkStatsLevel1(df1)

*** STATS *** Df Stats PlanSizeMB:9.864668,  EstimatorSizeMB:9.216484, PartitionNum: 4, PartitionSizeMB:2.466167, PartitionSizeMB:2.304121


prm1: PrmSparkStatsLevel1 = PrmSparkStatsLevel1@274e2fce


In [102]:
val prm2 = new PrmSparkStatsLevel2(df2)

*** STATS *** Df Stats PlanSizeMB:9.864668,  EstimatorSizeMB:9.027351, PartitionNum: 4, PartitionSizeMB:2.466167, PartitionSizeMB:2.2568378
*** STATS *** Df Partition record 
        count 1282382


prm2: PrmSparkStatsLevel2 = PrmSparkStatsLevel2@4b0b506b


In [104]:
val prm3 = new PrmSparkStatsLevel3(df3)

*** STATS *** Df Stats PlanSizeMB:9.864668,  EstimatorSizeMB:20.057, PartitionNum: 4, PartitionSizeMB:2.466167, PartitionSizeMB:5.01425
*** STATS *** Df Partition Info
+-----------+-------+
|cutoff_date|  count|
+-----------+-------+
| 2020-01-03|1282382|
+-----------+-------+



prm3: PrmSparkStatsLevel3 = PrmSparkStatsLevel3@2fed7552


In [105]:
val prm4 = new PrmSparkStatsLevel4(df4)

*** STATS *** Df Stats PlanSizeMB:9.864668,  EstimatorSizeMB:17.436272, PartitionNum: 4, PartitionSizeMB:2.466167, PartitionSizeMB:4.359068
*** STATS *** Df Partition record 
        count 1282382
*** STATS *** Df Partition Info
+-----------+-------+
|cutoff_date|  count|
+-----------+-------+
| 2020-01-03|1282382|
+-----------+-------+



prm4: PrmSparkStatsLevel4 = PrmSparkStatsLevel4@12eb4e22


In [106]:
val prm8 = new PrmSparkStatsLevel8(df8)

*** STATS *** Df Stats PlanSizeMB:9.864668,  EstimatorSizeMB:17.10936, PartitionNum: 4, PartitionSizeMB:2.466167, PartitionSizeMB:4.27734
*** STATS *** Show data:
+-------------------+-------------+-----------+------------------+--------------------+-----------+
|campaign_start_date|personal_type|personal_id|      segment_desc|   audtiminsert_date|cutoff_date|
+-------------------+-------------+-----------+------------------+--------------------+-----------+
|         2019-09-30|            L|   00081633|  08. Bajo ingreso|2019-11-07 19:12:...| 2020-01-03|
|         2019-09-30|            L|   00950462|  08. Bajo ingreso|2019-11-07 19:12:...| 2020-01-03|
|         2019-09-30|            L|   02446085|              None|2019-11-07 19:12:...| 2020-01-03|
|         2019-09-30|            L|   06674137|              None|2019-11-07 19:12:...| 2020-01-03|
|         2019-09-30|            L|   06694902|              None|2019-11-07 19:12:...| 2020-01-03|
|         2019-09-30|            L|  

prm8: PrmSparkStatsLevel8 = PrmSparkStatsLevel8@76cbd63b
